In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
import numpy as np
from keras.layers import LSTM,Dropout,Dense,Activation
from keras.models import Sequential,Model,load_model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from music21 import converter, instrument, note, chord,stream
import glob

In [ ]:
def import_data():
  notes = list()
  for file in glob.glob('/gdrive/My Drive/midi_songs/*.mid'):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if(parts):
      notes_to_parse = parts.parts[0].recurse()
    else:
      notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
      if(isinstance(element,note.Note)):
        notes.append(str(element.pitch))
      elif(isinstance(element,chord.Chord)):
        notes.append('.'.join(str(n) for n in element.normalOrder))
  return notes
      

In [ ]:
notes = import_data()
print(notes)

In [ ]:
def generate_input_for_network(notes,n_vocab):
  sequence_length = 100
  pitchnames = sorted(set(item for item in notes)) #Umquie vales in the training set
  # create a dictionary to map pitches to integers
  notes_to_int = dict((pitchnames[i], i) for i in range(len(pitchnames)))
  network_in_X = list()
  network_out_y = list()
  for i in range(0,len(notes)-sequence_length):
    sequence_in = notes[i:i+sequence_length]
    sequence_out = notes[i+sequence_length]
    network_in_X.append([notes_to_int[char] for char in sequence_in])
    network_out_y.append(notes_to_int[sequence_out])
  n_patterns = len(network_in_X)
  network_in_X = np.reshape(network_in_X,(n_patterns,sequence_length,1))
  network_in_X = network_in_X / float(n_vocab)
  network_out_y = to_categorical(network_out_y)
  return network_in_X,network_out_y

In [ ]:
notes = import_data()
n_vocab = len(set(notes))
network_in_X,network_out_y = generate_input_for_network(notes,n_vocab)

In [ ]:
def create_network(network_in_X,n_vocab):
  model = Sequential()
  model.add(LSTM(256,input_shape=(network_in_X.shape[1],network_in_X.shape[2]),return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512,return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(256))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
  return model

In [ ]:
model=create_network(network_in_X,n_vocab)

In [ ]:
def train_model(model,network_in_X,network_out_y):
  filepath = "/gdrive/My Drive/Intermediate_weights_music_gen_v3/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
  checkpoint = ModelCheckpoint(filepath,monitor='loss',verbose=0,save_best_only=True,mode='min')
  callbacks_list = [checkpoint]
  model.fit(network_in_X,network_out_y,epochs=100,batch_size=64, callbacks=callbacks_list)

In [ ]:
train_model(model,network_in_X,network_out_y)

In [ ]:
filepath = "/gdrive/My Drive/Intermediate_weights_music_gen_v3/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint_path = '/gdrive/My Drive/Intermediate_weights_music_gen_v3/weights-improvement-77-0.6529-bigger.hdf5'
model = load_model(checkpoint_path)
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]
model.fit(network_in_X, network_out_y, epochs=100, batch_size=64, callbacks=callbacks_list, initial_epoch=77)